In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

C:\Users\USER\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:
import numpy as np
import pandas as pd

In [45]:
from tensorflow.keras.datasets import mnist

In [46]:
# 모델 생성 함수
def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation = "relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10,activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

In [49]:
# 훈련/테스트 데이터 불러오기
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype('float32') /255
test_images = test_images.reshape((10000, 28*28)).astype('float32') /255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [51]:
#모델 및 손실함수, 옵티마이저, 평가지표, 손실평균값 준비

model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer=keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

### -------- 훈련 및 검증 메서드 사용자 정의 루프 활용 ---------

In [52]:
# 훈련 스텝 함수

def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss=loss_fn(targets, predictions)
    gradients=tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs["loss"] = loss_tracking_metric.result()
    return logs

In [53]:
# 지표 초기화

def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

In [54]:
# 훈련 실행 루프

training_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels))
training_dataset = training_dataset.batch(32)
epochs = 3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"{epoch}번째 에포크 결과")
    for key, value in logs.items():
        print(f"...{key} : {value:.4f}")

0번째 에포크 결과
...sparse_categorical_accuracy : 0.9143
...loss : 0.2905


KeyboardInterrupt: 

In [ ]:
# 검증 및 테스트 위한 스텝 함수

def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss=loss_fn(targets, predictions)
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
        
    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return.logs

In [ ]:
# 검증/테스트 수행
val_dataset = tf.data.Dataset.from_tensor_slices(
    (val_images, val_labels))
training_dataset = training_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("평가 결과 : ")
for key, value in logs.items():
    print(f"...{key} : {value:.4f}")

In [ ]:
# tf function 데코레이터 추가
@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss=loss_fn(targets, predictions)
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
        
    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return.logs

In [ ]:
# 데코레이터 추가 후 평가
val_dataset = tf.data.Dataset.from_tensor_slices(
    (val_images, val_labels))
training_dataset = training_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("평가 결과 : ")
for key, value in logs.items():
    print(f"...{key} : {value:.4f}")

### -------- fit() 메서드로 사용자 정의 루프 활용 ---------

In [ ]:
# Code 7-26
loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name="loss")

class CustomModel(keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        loss_tracker.update_state(loss)
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        return [loss_tracker]


In [ ]:
# page 277 학습 실행
inputs = keras.Input(shape=(28 * 28, ))
features = layers.Dense(512, activation="relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)